In [3]:
import numpy as np
import matplotlib.pyplot
import nnfs
from nnfs.datasets import spiral_data
from NNS import NeuralNetwork as NN #import neural net code from github to reduce copy/pasting

# Chapter 10

## Optimizers

### Stochastic Gradient Descent (SGD)

In [5]:
class Optimizer_SGD:
    
    # Initialize optimizer - set settings,
    # learning rate of 1.0 is set as a defaults
    def __init__(self, learning_rate=1.0):
        self.learning_rate = learning_rate
    
    # Update Parameters
    def update_params(self, layer):
        layer.weights += -self.learning_rate * layer.dweights
        layer.biases += -self.learning_rate * layer.dbiases

X, y = spiral_data(samples=100, classes=3)

dense1 = NN.Layer_Dense(2,64)
activation1 = NN.Activation_ReLU()
dense2 = NN.Layer_Dense(64,3)
loss_activation = NN.Activation_Softmax_Loss_CategoricalCrossentropy()
optimzer = Optimizer_SGD()

dense1.forward(X)
activation1.forward(X)
dense2.forward(activation1.output)
loss = loss_activation.forward(dense2.output,y)

print('loss: ', loss)



AttributeError: module 'NNS.NeuralNetwork' has no attribute 'Activation_Softmax_Loss_CategoricalCrossentropy'